<a href="https://colab.research.google.com/github/chihab216/Recommended_System/blob/master/Tp_khnifass_recommendedSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The dataset is avaible here : http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html

In [ ]:
bfrom google.colab import files
files.upload()

In [ ]:
import pandas as pd
ratings = pd.read_csv('ML_ratings.csv')


My_rating = pd.DataFrame({'userId': [(max(ratings.userId)+1) for i in range (10) ], 'movieId': [1,2,3,4,5,6,34,46,47,50],'rating': [2.0,2.0,3.0,4.0,3.0,3.3,3.4,4.6,4.7,5.0]})

ratings = ratings.append(My_rating, ignore_index=True)

ratings

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,movieId,rating,timestamp,userId
0,1,4.0,964982703.0,1
1,3,4.0,964981247.0,1
2,6,4.0,964982224.0,1
3,47,5.0,964983815.0,1
4,50,5.0,964982931.0,1
...,...,...,...,...
100841,6,3.3,NaN,611
100842,34,3.4,NaN,611
100843,46,4.6,NaN,611
100844,47,4.7,NaN,611


In [ ]:
pip install surprise

     |████████████████████████████████| 6.5MB 9.2MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678051 sha256=f2cfad00b5a3ea8e708f07a0282f0954584676a05bc875337696219441323a90
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [ ]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
import surprise
n_folds = 5

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader()

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

#Knnbaseline
res_KNNBaseline = cross_validate(surprise.prediction_algorithms.KNNBaseline(), data, cv=n_folds)
res_KNNBaseline = pd.DataFrame.from_dict(res_KNNBaseline)
mean = []
std = []
for col in res_KNNBaseline.columns:
  mean.append(res_KNNBaseline[col].mean())
  std.append(res_KNNBaseline[col].std())
res_KNNBaseline.loc["mean"] = mean
res_KNNBaseline.loc["std"] = std
print("KNNBASELINE")
print(res_KNNBaseline)
#svd
res_SVD = cross_validate(surprise.prediction_algorithms.matrix_factorization.SVD(), data, cv=n_folds)
res_SVD = pd.DataFrame.from_dict(res_SVD)
mean = []
std = []
for col in res_SVD.columns:
  mean.append(res_SVD[col].mean())
  std.append(res_SVD[col].std())
res_SVD.loc["mean"] = mean
res_SVD.loc["std"] = std

print("SVD") 
print(res_SVD)

res_df = pd.DataFrame([res_KNNBaseline['test_rmse'][0:5], res_SVD['test_rmse'][0:5]])
res_df = res_df.T
models = ['KNN_baseline', 'SVD']
res_df.columns = models
print("\nmean results:")
print(res_df.mean())

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
KNNBASELINE
      test_rmse  test_mae  fit_time  test_time
0      0.881030  0.670424  0.348493   2.009964
1      0.874955  0.669325  0.378736   2.039252
2      0.874977  0.666025  0.375595   2.004141
3      0.871343  0.667056  0.380049   1.960691
4      0.877102  0.673037  0.391603   2.017291
mean   0.875881  0.669173  0.374895   2.006268
std    0.003545  0.002780  0.015953   0.028744
SVD
      test_rmse  test_mae  fit_time  test_time
0      0.875904  0.672184  4.762560   0.2

In [ ]:




from __future__ import (absolute_import, division, print_function,

                        unicode_literals)

from collections import defaultdict



from surprise import SVD

from surprise import Dataset



##function which give the n better prediction for all users

def get_top_n(predictions, n):

    



    # First map the predictions to each user.

    top_n = defaultdict(list)

    for uid, iid, true_r, est, _ in predictions:

        top_n[uid].append((iid, est))



    # Then sort the predictions for each user and retrieve the k highest ones.

    for uid, user_ratings in top_n.items():

        user_ratings.sort(key=lambda x: x[1], reverse=True)

        top_n[uid] = user_ratings[:n]



    return top_n







In [ ]:
# First train an SVD algorithm on the movielens dataset.



trainset = data.build_full_trainset()

algo = SVD()

algo.fit(trainset)





testset = trainset.build_anti_testset()

predictions = algo.test(testset)



top_n = get_top_n(predictions, n=15)



# Print the recommended items for the user 611 ( me)

for uid, user_ratings in top_n.items():
  if (uid==611):
    ar = [[iid,est] for (iid,est) in user_ratings]
    df = pd.DataFrame(ar, index = [i for i in range (1,16)], columns = ['ItemId', 'estimate_rate'])
    print(df)

    ItemId  estimate_rate
1     1204       4.402524
2     2329       4.349658
3     1223       4.344793
4      951       4.313906
5      593       4.312332
6     4011       4.308728
7     1221       4.307863
8   112852       4.268614
9    99114       4.264717
10    5690       4.254372
11     318       4.253405
12     858       4.229904
13    1225       4.226985
14   48516       4.217584
15     914       4.201552


In [ ]:
# First train an KNN_baseline algorithm on the ratings dataset.

from surprise import KNNBaseline

trainset = data.build_full_trainset()

algo = KNNBaseline()

algo.fit(trainset)





testset = trainset.build_anti_testset()

predictions = algo.test(testset)



top_n = get_top_n(predictions, n=15)



# Print the recommended items for the user 611 ( me)

for uid, user_ratings in top_n.items():
  if (uid==611):
    ar = [[iid,est] for (iid,est) in user_ratings]
    df = pd.DataFrame(ar, index = [i for i in range (1,16)], columns = ['ItemId', 'estimate_rate'])
    print(df)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
    ItemId  estimate_rate
1     3851              5
2    59018              5
3    60943              5
4      391              5
5    26326              5
6   158966              5
7    92494              5
8   102217              5
9    67618              5
10    5867              5
11   81535              5
12      53              5
13    6402              5
14    8238              5
15   25887              5
